In [1]:
import json
import random

def generate_departments():
    departments = [
        {"id": 1, "name": "Software Engineering", "college": "Computing and Information Science"},
        {"id": 2, "name": "Computer Science", "college": "Computing and Information Science"},
        {"id": 3, "name": "Cybersecurity", "college": "Computing and Information Science"},
        {"id": 4, "name": "Biology", "college": "Natural Sciences"},
        {"id": 5, "name": "Microbiology", "college": "Natural Sciences"},
        {"id": 6, "name": "Science Laboratory Technology", "college": "Natural Sciences"},
        {"id": 7, "name": "Mathematics", "college": "Natural Sciences"},
        {"id": 8, "name": "Physics", "college": "Natural Sciences"},
        {"id": 9, "name": "Economics", "college": "Social Sciences"},
        {"id": 10, "name": "Accounting", "college": "Social Sciences"},
    ]
    return departments

def generate_courses():
    levels = [100, 200, 300, 400]
    courses = []
    departments = generate_departments()

    combined_courses = [
        {"course_code": "Math101", "course_name": "Basic Mathematics", "level": 100, "department_ids": [1, 2, 3]},
        {"course_code": "Math102", "course_name": "Advanced Mathematics", "level": 200, "department_ids": [4, 5, 6]},
        {"course_code": "GSP101", "course_name": "General Studies: Communication Skills", "level": 100, "department_ids": [1, 2, 3, 4, 5, 6]},
    ]

    for dept in departments:
        for level in levels:
            num_courses = 0

            # Determine number of courses based on level
            if level == 100:
                num_courses = random.randint(10, 14)
            elif level == 200:
                num_courses = random.randint(10, 14)
            elif level == 300:
                num_courses = random.randint(7, 10)
            elif level == 400:
                num_courses = random.randint(4, 6)

            # Generate departmental courses
            for i in range(1, num_courses + 1):
                courses.append({
                    "course_code": f"{dept['name'][:3].upper()}{level}{i:02d}",
                    "course_name": f"Course {i} for {dept['name']} Level {level}",
                    "department_id": dept["id"],
                    "level": level
                })

    # Add combined courses
    courses.extend(combined_courses)

    return courses

def export_data():
    data = {
        "departments": generate_departments(),
        "courses": generate_courses()
    }
    with open("exam_data_with_combined.json", "w") as file:
        json.dump(data, file, indent=4)
    print("Data exported to exam_data_with_combined.json")

export_data()


Data exported to exam_data_with_combined.json


In [2]:
import json
from datetime import datetime, timedelta

# Load data from JSON
def load_data(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

# Generate timetable
def generate_timetable(data, start_date, end_date):
    timetable = []
    courses = data["courses"]
    departments = data["departments"]

    # Generate dates for the timetable
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    current_date = start_date
    exam_times = [("08:00", "10:00"), ("10:30", "12:30"), ("14:00", "16:00"), ("16:30", "18:30")]

    # Initialize venues and invigilators
    venues = [{"id": i + 1, "name": f"Venue {i + 1}", "capacity": 50 + i * 10} for i in range(29)]
    invigilators = [{"id": i + 1, "name": f"Invigilator {i + 1}"} for i in range(50)]

    # Assign exams
    for level in [100, 200, 300, 400]:
        for dept in departments:
            # Filter courses for this department and level
            dept_courses = [course for course in courses if course["level"] == level and course.get("department_id") == dept["id"]]
            combined_courses = [course for course in courses if course["level"] == level and "department_ids" in course and dept["id"] in course["department_ids"]]
            all_courses = dept_courses + combined_courses

            for course in all_courses:
                # Assign date, time, venue, and invigilator
                for time_slot in exam_times:
                    if current_date <= end_date:
                        venue = find_available_venue(venues, course)
                        invigilator = find_available_invigilator(invigilators, current_date, time_slot)
                        
                        if venue and invigilator:
                            timetable.append({
                                "date": current_date.strftime("%Y-%m-%d"),
                                "day": current_date.strftime("%A"),
                                "start_time": time_slot[0],
                                "end_time": time_slot[1],
                                "course_code": course["course_code"],
                                "course_name": course["course_name"],
                                "level": course["level"],
                                "department": dept["name"] if "department_id" in course else "Combined",
                                "venue": venue["name"],
                                "invigilator": invigilator["name"],
                                "exam_type": "Written" if "department_id" in course else "CBT",
                            })
                            break

                # Move to the next day
                current_date += timedelta(days=1)
                if current_date.strftime("%H:%M") == "13:00":  # Skip lunch break
                    current_date += timedelta(hours=1)

    return timetable

# Find available venue
def find_available_venue(venues, course):
    for venue in venues:
        if venue["capacity"] >= 50:  # Example constraint
            return venue
    return None

# Find available invigilator
def find_available_invigilator(invigilators, date, time_slot):
    for invigilator in invigilators:
        return invigilator
    return None

# Save timetable to JSON
def save_timetable(timetable, file_path):
    with open(file_path, "w") as file:
        json.dump(timetable, file, indent=4)
    print(f"Timetable saved to {file_path}")

# Main function
def main():
    data = load_data("exam_data_with_combined.json")
    start_date = "2024-01-01"
    end_date = "2024-01-14"
    timetable = generate_timetable(data, start_date, end_date)
    save_timetable(timetable, "generated_timetable.json")

if __name__ == "__main__":
    main()


Timetable saved to generated_timetable.json


: 